In [ ]:
from neighborhoods import *
import time
import numpy as np

In [ ]:
# Path of the file
file_path = "../data/indoor_scan.ply"

# Load point cloud
data = read_ply(file_path)

# Concatenate data
points = np.vstack((data["x"], data["y"], data["z"])).T



# Question 3 

In [ ]:
neighbors_num = 100
radius = 0.2
num_queries = 10

# Pick random queries
random_indices = np.random.choice(points.shape[0], num_queries, replace=False)
queries = points[random_indices, :]

In [ ]:
# Search spherical
t0 = time.time()
neighborhoods = brute_force_spherical(queries, points, radius)
t1 = time.time()
print("KNN?", neighbors_num)
# Search KNN
neighborhoods = brute_force_KNN(queries, points, neighbors_num)
t2 = time.time()

# Print timing results
print(
    "{:d} spherical neighborhoods computed in {:.3f} seconds".format(
        num_queries, t1 - t0
    )
)
print("{:d} KNN computed in {:.3f} seconds".format(num_queries, t2 - t1))

# Time to compute all neighborhoods in the cloud
total_spherical_time = points.shape[0] * (t1 - t0) / num_queries
total_KNN_time = points.shape[0] * (t2 - t1) / num_queries
print(
    "Computing spherical neighborhoods on whole cloud : {:.0f} hours".format(
        total_spherical_time / 3600
    )
)
print("Computing KNN on whole cloud : {:.0f} hours".format(total_KNN_time / 3600))

# Question 4 a

In [ ]:
num_queries = 10
from scipy.spatial import cKDTree

num_neighbors = 1000000  #! really big
random_indices = np.random.choice(points.shape[0], num_queries, replace=False)
queries = points[random_indices, :]
radius = 0.2
num_queries = 1000
best_leaf_size = 128
print("search spherical with same parameters")
for leaf_size in [1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024]:
    kdtree = cKDTree(points, leafsize=leaf_size)

    t1 = time.time()

    # Search KNN
    distances, eighbors = kdtree.query(
        queries, k=num_neighbors, distance_upper_bound=radius
    )

    t2 = time.time()
    print("leaf size", leaf_size)
    print("KNN computed in {:.3f} seconds".format(t2 - t1))

## 4.a 2

In [ ]:
print("search KNN with best parameters")

radius = 0.2
num_queries = 1000
best_leaf_size = 128
random_indices = np.random.choice(points.shape[0], num_queries, replace=False)
queries = points[random_indices, :]
t1 = time.time()
# select random num_queries queries

# Search KNN
kdtree = cKDTree(points, leafsize=best_leaf_size)
distances, eighbors = kdtree.query(
    queries, k=neighbors_num, distance_upper_bound=radius
)
t2 = time.time()
print("leaf size", best_leaf_size)
print("KNN computed in {:.3f} seconds with 1000 random queries ".format(t2 - t1))

# Question 4.b 

In [ ]:
# Benchmark with radius
neighbors_num = 50000
num_queries = 1000
best_leaf_size = 128
random_indices = np.random.choice(points.shape[0], num_queries, replace=False)
queries = points[random_indices, :]
kdtree = cKDTree(points, leafsize=best_leaf_size)
print("start benchmark radius")
for radius in np.logspace(0, 2, 20, base=10):
    t1 = time.time()
    # Search KNN
    distances, eighbors = kdtree.query(
        queries, k=neighbors_num, distance_upper_bound=radius
    )

    t2 = time.time()
    print("radius", radius)
    print("KNN computed in {:.3f} seconds".format(t2 - t1))
# distances,eighbors=kdtree.query(points[:num_queries],k=neighbors_num,distance_upper_bound=radius)

In [ ]:
print("start computing for 20cm back")
num_queries = 100000
num_neighbors = 10000  #! really big
random_indices = np.random.choice(points.shape[0], num_queries, replace=False)
queries = points[random_indices, :]
radius = 0.2
num_queries = 1000
best_leaf_size = 128

t1 = time.time()
# Search KNN
distances, eighbors = kdtree.query(
    queries, k=num_neighbors, distance_upper_bound=radius
)
t2 = time.time()

# Print timing results
print(
    "{:d} KNN computed back with the best leaves  in {:.3f} seconds".format(
        num_queries, t2 - t1
    )
)

# Time to compute all neighborhoods in the cloud
total_KNN_time = points.shape[0] * (t2 - t1) / num_queries
print("Computing KNN on whole cloud : {:.0f} hours".format(total_KNN_time / 3600))

# Question Bonus: implement voxel_grid

In [ ]:
from scipy.spatial import cKDTree
import numpy as np


def grid_voxel(data, voxel_size=5):
    points = np.vstack((data["x"], data["y"], data["z"])).T
    colors = np.vstack((data["red"], data["green"], data["blue"])).T

    # Quantize the points to voxel grid
    voxel_indices = np.floor(points / voxel_size).astype(np.int32)

    # Use a dictionary to aggregate points and colors within each voxel
    voxel_dict = {}
    for i, voxel_idx in enumerate(map(tuple, voxel_indices)):
        if voxel_idx not in voxel_dict:
            voxel_dict[voxel_idx] = {"points": [], "colors": []}
        voxel_dict[voxel_idx]["points"].append(points[i])
        voxel_dict[voxel_idx]["colors"].append(colors[i])

    # Compute the average position and color for each voxel
    subsampled_points = []
    subsampled_colors = []
    for voxel in voxel_dict.values():
        avg_point = np.mean(voxel["points"], axis=0)
        avg_color = np.mean(voxel["colors"], axis=0)
        # convert avg_color to int
        avg_color = avg_color.astype(np.uint8)

        subsampled_points.append(avg_point)
        subsampled_colors.append(avg_color)

    return np.array(subsampled_points), np.array(subsampled_colors)

In [ ]:
file_path = "../data/indoor_scan.ply"
data = read_ply(file_path)

In [ ]:
# Concatenate x, y, and z in a (N*3) point matrix
points = np.vstack((data["x"], data["y"], data["z"])).T

# Concatenate R, G, and B channels in a (N*3) color matrix
colors = np.vstack((data["red"], data["green"], data["blue"])).T
transformed_points, transformed_colors = grid_voxel(data, 0.4)

# Save point cloud
# *********************
#
#   Save your result file
#   (See write_ply function)
#


# Save point cloud
write_ply(
    "../decimated_indoor_scan.ply",
    [transformed_points, transformed_colors],
    ["x", "y", "z", "red", "green", "blue"],
)

print("Done")